In [82]:
import torch
import torch.utils.data as data
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torchvision import datasets,models,transforms

import sys
sys.path.append('/mnt/workspace/pytorch-captcha-recognition/')
print (sys.path)
import my_dataset
import captcha_setting
import importlib
importlib.reload(my_dataset)

NUM_EPOCH=10
batch_size=64
lr = 0.001
device = torch.device('cuda:0')
DEVICE = 'cuda:0'

print(captcha_setting.CLASS_NUM)
#help(models.resnet.ResNet)

['/mnt/workspace', '/opt/conda/lib/python310.zip', '/opt/conda/lib/python3.10', '/opt/conda/lib/python3.10/lib-dynload', '', '/opt/conda/lib/python3.10/site-packages', '.', '.', 'mnt/workspace/pytorch-captcha-recognition/', 'mnt/workspace/pytorch-captcha-recognition/', 'mnt/workspace/pytorch-captcha-recognition/', '/mnt/workspace/pytorch-captcha-recognition/', '/mnt/workspace/pytorch-captcha-recognition/', '/mnt/workspace/pytorch-captcha-recognition/', '/mnt/workspace/pytorch-captcha-recognition/', '/mnt/workspace/pytorch-captcha-recognition/', '/mnt/workspace/pytorch-captcha-recognition/', '/mnt/workspace/pytorch-captcha-recognition/', '/mnt/workspace/pytorch-captcha-recognition/', '/mnt/workspace/pytorch-captcha-recognition/', '/mnt/workspace/pytorch-captcha-recognition/', '/mnt/workspace/pytorch-captcha-recognition/', '/mnt/workspace/pytorch-captcha-recognition/', '/mnt/workspace/pytorch-captcha-recognition/', '/mnt/workspace/pytorch-captcha-recognition/', '/mnt/workspace/pytorch-ca

In [84]:
torch.manual_seed(1)

model = models.resnet18(captcha_setting.CLASS_NUM)
model.to('cuda:0')

optimizer = torch.optim.Adam(model.parameters(), lr=lr)

def compute_accuracy(model, data_loader, device):
    correct_pred, num_examples = 0, 0
    for i, (features, targets) in enumerate(data_loader):
            
        features = features.to(device)
        targets = targets.to(device)

        logits, probas = model(features)
        _, predicted_labels = torch.max(probas, 1)
        num_examples += targets.size(0)
        correct_pred += (predicted_labels == targets).sum()
    return correct_pred.float()/num_examples * 100
    

start_time = time.time()
train_loader = my_dataset.get_train_data_loader() 
for epoch in range(NUM_EPOCH):
    
    model.train()
    for batch_idx, (features, targets) in enumerate(train_loader):
        
        features = features.to(DEVICE)
        targets = targets.to(DEVICE)
        print(features.size(),targets.size())
            
        ### FORWARD AND BACK PROP
        logits, probas = model(features)
        cost = F.cross_entropy(logits, targets)
        optimizer.zero_grad()
        
        cost.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ### LOGGING
        if not batch_idx % 50:
            print ('Epoch: %03d/%03d | Batch %04d/%04d | Cost: %.4f' 
                   %(epoch+1, NUM_EPOCHS, batch_idx, 
                     len(train_loader), cost))

        

    model.eval()
    with torch.set_grad_enabled(False): # save memory during inference
        print('Epoch: %03d/%03d | Train: %.3f%%' % (
              epoch+1, NUM_EPOCHS, 
              compute_accuracy(model, train_loader, device=DEVICE)))
        
    print('Time elapsed: %.2f min' % ((time.time() - start_time)/60))
    
print('Total Training Time: %.2f min' % ((time.time() - start_time)/60))

torch.Size([3, 50, 160]) 144
torch.Size([3, 50, 160]) 144
torch.Size([3, 50, 160]) 144
torch.Size([3, 50, 160]) 144
torch.Size([3, 50, 160]) 144
torch.Size([3, 50, 160]) 144
torch.Size([3, 50, 160]) 144
torch.Size([3, 50, 160]) 144
torch.Size([3, 50, 160]) 144
torch.Size([3, 50, 160]) 144
torch.Size([3, 50, 160]) 144
torch.Size([3, 50, 160]) 144
torch.Size([3, 50, 160]) 144
torch.Size([3, 50, 160]) 144
torch.Size([3, 50, 160]) 144
torch.Size([3, 50, 160]) 144
torch.Size([3, 50, 160]) 144
torch.Size([3, 50, 160]) 144
torch.Size([3, 50, 160]) 144
torch.Size([3, 50, 160]) 144
torch.Size([3, 50, 160]) 144
torch.Size([3, 50, 160]) 144
torch.Size([3, 50, 160]) 144
torch.Size([3, 50, 160]) 144
torch.Size([3, 50, 160]) 144
torch.Size([3, 50, 160]) 144
torch.Size([3, 50, 160]) 144
torch.Size([3, 50, 160]) 144
torch.Size([3, 50, 160]) 144
torch.Size([3, 50, 160]) 144
torch.Size([30, 3, 50, 160]) torch.Size([30, 144])


ValueError: too many values to unpack (expected 2)

In [75]:
def train_model(model, train_data_loader,crtiation, optimizer,schedular, num_epochs=NUM_EPOCH):
    
    begin_time = time.time()
    arr_acc = [] # 用于作图
 
    for epoch in range(num_epochs):
        print("-*-" * 20)
        item_acc = []
        schedular.step()
        model.train()
        running_loss = 0.0
        running_acc = 0.0
        for images, labels in train_data_loader:
            #print(images.size(),labels.size())
            images.to(device)
            labels.to(device)
            optimizer.zero_grad()

            opt = model(images.cuda())
            # opt = model(images)
            _,pred = torch.max(opt,1)
            #print(pred,labels,pred.size(),labels.size())
            labels = labels.cuda()
            loss = crtiation(opt, labels)
            loss.backward()
            optimizer.step()
                        
            running_loss += loss.item()*images.size(0)
            print(pred,labels,pred.size(),labels.size())
            #running_acc += torch.sum(pred==labels)
        epoch_loss = running_loss
        #epoch_acc = running_acc.double()
        print('epoch={}, Loss={:.4f},'.format(epoch, epoch_loss, ))
        #item_acc.append(epoch_acc)
 
        #arr_acc.append(item_acc)
        
        
    time_elapes = time.time() - begin_time
    print('Training Complete in{:.0f}m {:0f}s'.format(
        time_elapes // 60, time_elapes % 60
    ))
    
    return model#,arr_acc

In [76]:
def train():
    model_resnet = models.resnet18(pretrained=True)
    num_fits = model_resnet.fc.in_features
    model_resnet.fc = nn.Linear(num_fits,captcha_setting.CLASS_NUM)
    model_resnet = model_resnet.to(device)
    model_resnet.cuda()
    
    criterion = nn.CrossEntropyLoss()
    optimizer_resnet = optim.SGD(model_resnet.parameters(),lr=lr,momentum=0.9)
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer_resnet,step_size=10,gamma=0.1)
    train_data_loader = my_dataset.get_train_data_loader()    
    model_resnet = train_model(model_resnet,train_data_loader,criterion,optimizer_resnet,exp_lr_scheduler,NUM_EPOCH)
    
    torch.save(model_resnet.state_dict(),'./capReg.pth')
    
train()

-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
torch.Size([3, 50, 160]) 144
torch.Size([3, 50, 160]) 144
torch.Size([3, 50, 160]) 144
torch.Size([3, 50, 160]) 144
torch.Size([3, 50, 160]) 144
torch.Size([3, 50, 160]) 144
torch.Size([3, 50, 160]) 144
torch.Size([3, 50, 160]) 144
torch.Size([3, 50, 160]) 144
torch.Size([3, 50, 160]) 144
torch.Size([3, 50, 160]) 144
torch.Size([3, 50, 160]) 144
torch.Size([3, 50, 160]) 144
torch.Size([3, 50, 160]) 144
torch.Size([3, 50, 160]) 144
torch.Size([3, 50, 160]) 144
torch.Size([3, 50, 160]) 144
torch.Size([3, 50, 160]) 144
torch.Size([3, 50, 160]) 144
torch.Size([3, 50, 160]) 144
torch.Size([3, 50, 160]) 144
torch.Size([3, 50, 160]) 144
torch.Size([3, 50, 160]) 144
torch.Size([3, 50, 160]) 144
torch.Size([3, 50, 160]) 144
torch.Size([3, 50, 160]) 144
torch.Size([3, 50, 160]) 144
torch.Size([3, 50, 160]) 144
torch.Size([3, 50, 160]) 144
torch.Size([3, 50, 160]) 144
tensor([ 93, 104, 133, 133,  96,  36,  96,  96, 122,  60,  30,  74,  

In [56]:
import os
print(os.getcwd())

/mnt/workspace


In [ ]:
num_epochs = 30
batch_size = 100
learning_rate = 0.001

def cnn_main():
    cnn = CNN()
    cnn.train()
    print('init net')
    criterion = nn.MultiLabelSoftMarginLoss()
    optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate)

    # Train the Model
    train_dataloader = my_dataset.get_train_data_loader()
    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(train_dataloader):
            images = Variable(images)
            labels = Variable(labels.float())
            predict_labels = cnn(images)
            # print(predict_labels.type)
            # print(labels.type)
            loss = criterion(predict_labels, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if (i+1) % 10 == 0:
                print("epoch:", epoch, "step:", i, "loss:", loss.item())
            if (i+1) % 100 == 0:
                torch.save(cnn.state_dict(), "./model.pkl")   #current is model.pkl
                print("save model")
        print("epoch:", epoch, "step:", i, "loss:", loss.item())
    torch.save(cnn.state_dict(), "./model.pkl")   #current is model.pkl
    print("save last model")

In [ ]:
def train_model(model, crtiation, optimizer,schedular, num_epochs=NUM_EPOCH):

    train_dataloader = my_dataset.get_train_data_loader()
    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(train_dataloader):

In [ ]:



def train():
    model_resnet = models.resnet18(pretrained=True)
    num_fits = model.fc.in_features
    model_resnet.fc = nn.Linear(num_fits,captcha_setting.CLASS_NUM)
    model_resnet = model_resnet.to(device)
    model_resnet.cuda()
    
    criterion = nn.CrossEntropyLoss()
    optimizer_resnet = optim.SGD(model_resnet.parameters(),lr=lr,momentum=0.9)
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer_resnet,step_size=10,gamma=0.1)
    train_data_loader = my_dataset.get_train_data_loader()    
    model_resnet,acc = train_model(model_resnet,train_data_loader,criterion,optimizer_resnet,exp_lr_scheduler,NUM_EPOCH)
    
    torch.save(model_ft.state_dict(),'./model/capReg.pth')
    

In [2]:
import os
import torch.utils.data as data
import torch
import torch.optim as optim
import torch.nn as nn
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
from PIL import Image
import time
import copy
#import pandas as pd
#import matplotlib.pyplot as plt
import numpy as np
#%matplotlib inline

NUM_EPOCH = 100 # 默认迭代次数
batch_size = 64
device = torch.device('cuda:0') # 默认使用 GPU
NUMCLASS = 4 # 类别数

def default_loader(path):
    with open(path, 'rb') as f:
        with Image.open(f) as img:
            return img.convert('RGB')

class CustomImageLoader(data.Dataset): # 定义自己的数据类
    ##自定义类型数据输入
    def __init__(self, img_path, txt_path, dataset = '', data_transforms=None, loader = default_loader):
        im_list = []
        im_labels = []
        with open(txt_path, 'r') as files:
            for line in files:
                #/x/y/a.jpg 1
                #/x/y/b.jpg 2
                items = line.split()
                im_list.append(items[0])
                im_labels.append(int(items[1]))
        self.imgs = im_list
        self.labels = im_labels
        self.data_tranforms = data_transforms
        self.loader = loader
        self.dataset = dataset
 
    def __len__(self):
        return len(self.imgs)
 
    def __getitem__(self, item):
        img_name = self.imgs[item]
        label = self.labels[item]
        img = self.loader(img_name)
 
        if self.data_tranforms is not None:
            try:
                img = self.data_tranforms[self.dataset](img)
            except:
                print("Cannot transform image: {}".format(img_name))
        return img, label
    
data_tranforms={
    'Train':transforms.Compose([
        transforms.RandomResizedCrop(50), # 随机裁剪为不同的大小和宽高比,缩放所为制定的大小
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225]) # 各通道颜色的均值和方差,用于归一化
    ]),
    'Test':transforms.Compose([
        transforms.Resize(64), # 变换大小
        transforms.CenterCrop(50), # 中心裁剪
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
    ])
}
 
image_datasets = {x : CustomImageLoader('/', # 默认目录为根目录，配搭文件中使用全路径
                                        txt_path=('./data/{0}.txt'.format(x)), # 标签文件
                                        data_transforms=data_tranforms,
                                        dataset=x) for x in ['Train', 'Test']
                  }
 
dataloders = {x: torch.utils.data.DataLoader(image_datasets[x],
                                                 batch_size=batch_size,
                                                 shuffle=True) for x in ['Train', 'Test']}
 
dataset_sizes = {x: len(image_datasets[x]) for x in ['Train', 'Test']} # 数据大小

def train_model(model, crtiation, optimizer,schedular, num_epochs=NUM_EPOCH):
    
    begin_time = time.time()
    best_weights = copy.deepcopy(model.state_dict())#copy the weights from the model
    best_acc = 0.0
    arr_acc = [] # 用于作图
 
    for epoch in range(num_epochs):
        print("-*-" * 20)
        item_acc = []
        for phase in ['Train', 'Test']:
            if phase=='Train':
                schedular.step()
                model.train()
            else:
                model.eval()
            running_loss = 0.0
            running_acc = 0.0
 
            for images, labels in dataloders[phase]:
                images.to(device)
                labels.to(device)
                optimizer.zero_grad()
 
                with torch.set_grad_enabled(phase=='Train'):
                    opt = model(images.cuda())
                    # opt = model(images)
                    _,pred = torch.max(opt,1)
                    labels = labels.cuda()
                    loss = crtiation(opt, labels)
                    if phase=='Train':
                        loss.backward()
                        optimizer.step()
 
                running_loss += loss.item()*images.size(0)
                running_acc += torch.sum(pred==labels)
            epoch_loss = running_loss/dataset_sizes[phase]
            epoch_acc = running_acc.double()/dataset_sizes[phase]
            print('epoch={}, Phase={}, Loss={:.4f}, ACC:{:.4f}'.format(epoch, phase, 
                                                                       epoch_loss, epoch_acc))
            item_acc.append(epoch_acc)
 
            if phase == 'Test' and epoch_acc>best_acc:
                # Upgrade the weights
                best_acc=epoch_acc
                best_weights = copy.deepcopy(model.state_dict())
        arr_acc.append(item_acc)
        
    time_elapes = time.time() - begin_time
    print('Training Complete in{:.0f}m {:0f}s'.format(
        time_elapes // 60, time_elapes % 60
    ))
    print('Best Val ACC: {:}'.format(best_acc))
 
    model.load_state_dict(best_weights) # 保存最好的参数
    return model,arr_acc

if __name__ == '__main__':
    
    model_ft = models.resnet18(pretrained=True)
    num_fits = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_fits, NUMCLASS) # 替换最后一个全连接层
    model_ft = model_ft.to(device)
    model_ft.cuda()
    criterion = nn.CrossEntropyLoss()
    optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)
    train_data_loader = my_dataset.get_train_data_loader()    
    model_ft,arr_acc = train_model(model_ft,train_data_loader, criterion, optimizer_ft, exp_lr_scheduler, NUM_EPOCH)
    ## 保存模型 
    torch.save(model_ft.state_dict(), './model/my_model.pth')

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


FileNotFoundError: [Errno 2] No such file or directory: './data/Train.txt'